General outline:
0. Constants:
    * Multiplier board

* Helper functions

* make functions that define the area that the word is in on the board (given a letter?)
    * allow another function to tell what the letters are in that area.


Main steps needed:
1. Take in an array of the board and your hand letters
2. Tell the available permutations and their locations on the board
    * return a dictionary with the word as a key and the location as the value
3. See what crossing words those permutations are going to make
    * this will return a dictionary of the main word and its position as a key and the crossing words as the values
4. Check to see if the permutations and their crossing words are in the dictionary
    * take in the dictinary from above?
For Step 2:
1. Need to get permutations for each row
       * Options:
           1. Get permutations around each group of letters like I've been doing
           2. Look at it on a row-wide basis
               * Need to make sure all the sample letters stay together
    

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import csv
import itertools
import re


<IPython.core.display.Javascript object>

In [3]:
sample_board = [
    ["", "", "", "", "", "", "", "", "", "", "", "e", "", "", ""],  # 0
    ["", "", "", "", "", "", "", "", "", "z", "o", "n", "a", "", "w"],  # 1
    ["", "", "", "", "", "", "", "", "", "", "", "a", "", "", "i"],  # 2
    ["", "", "", "", "", "", "", "", "", "r", "i", "m", "a", "", "l"],  # 3
    ["", "", "", "", "", "", "", "", "", "", "", "o", "b", "e", "y"],  # 4
    ["", "", "", "", "", "", "", "", "", "j", "u", "r", "a", "l", ""],
    ["", "", "", "", "", "", "", "b", "e", "e", "p", "", "", "", ""],  # 6
    ["", "", "", "", "", "", "y", "e", "w", "s", "", "t", "", "", ""],
    ["", "", "", "", "", "", "", "l", "e", "t", "c", "h", "", "", ""],  # 8
    ["", "", "", "", "h", "u", "r", "l", "", "", "", "i", "", "r", ""],
    ["", "", "", "", "", "", "e", "", "", "", "", "g", "", "e", ""],  # 10
    ["", "", "", "", "", "", "d", "", "", "", "", "h", "u", "e", "d"],
    ["", "", "", "", "", "", "", "", "", "", "", "", "", "f", ""],  # 12
    ["", "", "", "", "", "", "", "", "", "v", "e", "e", "p", "s", ""],
    ["", "", "", "", "", "", "", "", "n", "u", "n", "", "", "", ""],  # 14
]
sample_board_letters = ["o", "d", "a", "s"]  # "d", "d", "a", "r"]

# numpy of the board
board_array = np.array(sample_board)

# making the empty strings spaces
neat_sample_array = np.copy(board_array)
neat_sample_array[neat_sample_array == ""] = " "


<IPython.core.display.Javascript object>

In [247]:
class Scrabbler:
    data_path = "../sowpods.txt"
    with open(data_path, newline="") as f:
        reader = csv.reader(f)
        allwords = list(reader)
    WORDLIST = list(itertools.chain.from_iterable(allwords))
    WORDLIST = [x.lower() for x in WORDLIST]
    WORDSET = set(WORDLIST)
    BOARD_LENGTH = 15
    MAX_I=14

    LETTER_MULTIPLIERS = [
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 0
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],  # 1
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 2
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 3
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 4
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 6
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 8
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 10
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 12
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 14
    ]
    LETTER_MULTIPLIERS_ARRAY = np.array(LETTER_MULTIPLIERS)

    WORD_MULTIPLIERS = [
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 0
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],  # 1
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 2
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],  # 3
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 4
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 6
        [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 8
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 10
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 12
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 14
    ]
    WORD_MULTIPLIERS_ARRAY = np.array(WORD_MULTIPLIERS)

    TILE_SCORES = {
        "a": 1,
        "b": 4,
        "c": 4,
        "d": 2,
        "e": 1,
        "f": 4,
        "g": 3,
        "h": 3,
        "i": 1,
        "j": 10,
        "k": 5,
        "l": 2,
        "m": 4,
        "n": 2,
        "o": 1,
        "p": 4,
        "q": 10,
        "r": 1,
        "s": 1,
        "t": 1,
        "u": 2,
        "v": 5,
        "w": 4,
        "x": 8,
        "y": 3,
        "z": 10,
    }

    def __init__(self, board, hand_letters):
        self.board = board
        self.hand_letters = hand_letters

    # Navigator functions
    def check_yx(self, y, x):
        if x>-1 and x<15 and y>-1 and y<15:
            return bool(self.board[y,x])
        else:
            return 'out'
    
    def check_right(self, y, x):
        if x < self.BOARD_LENGTH - 1:
            return bool(self.board[y, x + 1]), self.board[y, x + 1]
        else:
            return "out"

    def check_left(self, y, x):
        if x > 0:
            return bool(self.board[y, x - 1]), self.board[y, x - 1]
        else:
            return "out"

    def check_above(self, y, x):
        if y > 0:
            return bool(self.board[y - 1, x]), self.board[y - 1, x]
        else:
            return "out"

    def check_below(self, y, x):
        if y < self.BOARD_LENGTH - 1:
            return bool(self.board[y + 1, x]), self.board[y + 1, x]
        else:
            return "out"
        
    # For potential words of a given length
    #start at the left, travel right (keep track of empties) till you hit a letter
    def hand_letter_permutations(self, length):
        #this returns a list of permutations of the hand letters of a given length
        letter_permutations= list(itertools.permutations(self.hand_letters, length))
        return letter_permutations

    def row_permutations(self, row):
        y=row
        board_row=self.board[row]
        no_hand_letters= len(self.hand_letters)
        no_pre_filled=sum(list(map(bool,board_row)))
        row_perms_dict={}
        if no_pre_filled>0:
            for perm_len in range(1, no_hand_letters+1):
                hand_perms=self.hand_letter_permutations(perm_len)
                #for x in range(0, BOARD_LENGTH-(len(self.hand_letters)-1)-no_pre_filled):
                #x=0
                no_tiles_to_right=no_pre_filled
                for hand_perm in hand_perms:
                #while x + no_tiles_to_right+perm_len<self.BOARD_LENGTH:
                    x=0
                    if self.check_yx(y,x)==False: #and sum(list(map(bool,board_row[x:x+no_hand_letters])))
                        #for hand_perm in hand_perms:
                        while True:
                            complete_word, word_y, word_x=self.complete_word_y_x(row, x, hand_perm)
                            if complete_word==[]:
                                break
                            elif len(complete_word)>perm_len:
                                if complete_word in row_perms_dict and [word_y,word_x] not in row_perms_dict[complete_word]:
                                    row_perms_dict[complete_word].append([word_y,word_x])
                                else:
                                    row_perms_dict[complete_word]=[word_y,word_x]
                            x=x+1
                    #no_tiles_to_right=sum(list(map(bool,board_row[x+1:])))#TODO: +1 here?
        return row_perms_dict

    #fill the next n spaces with hand tiles and find the surrounding word
    def complete_word_y_x(self, row, x, hand_perm):
        hand_perm=list(hand_perm)
        perm_len=len(hand_perm)
        X=x
        board_row=self.board[row]
        y=row
        no_filled=0
        filled_tiles=[]
        left_letters=[]
        right_letters=[]
        while self.check_left(y,x)[0]==True:
            x=x-1
            left_letters=left_letters+[board_row[x]]
        left_letters=left_letters[::-1]
        x=X
        while no_filled<perm_len:
            if self.check_yx(y,x)==True:
                filled_tiles=filled_tiles+[board_row[x]]
            elif self.check_yx(y,x)=='out':
                return [],[],[] #TODO: check if this is right. Essentially this means you passed in too long a perm
            elif self.check_yx(y,x)==False:
                new_tile=hand_perm.pop(0)
                filled_tiles=filled_tiles+[new_tile]
                no_filled=no_filled+1
            x=x+1
        x=x-1
        while self.check_right(y,x)[0]==True:
            x=x+1
            right_letters=right_letters+[board_row[x]]
        complete_word_as_list=left_letters+filled_tiles+right_letters
        word_x=X-len(left_letters)
        complete_word=''.join(complete_word_as_list)
        return complete_word, y, word_x


<IPython.core.display.Javascript object>

In [248]:
# for edge row, I could find permutations of the surrounding area including spaces

<IPython.core.display.Javascript object>

In [249]:
sam = Scrabbler(board_array, sample_board_letters)


<IPython.core.display.Javascript object>

In [250]:
sam.complete_word_y_x(7, 12, ("a", "b", "c", "d"))

([], [], [])

<IPython.core.display.Javascript object>

In [251]:
sam.row_permutations(7)

{'oyews': [7, 5],
 'yewsot': [7, 6],
 'to': [7, 11, [7, 11]],
 'dyews': [7, 5],
 'yewsdt': [7, 6],
 'td': [7, 11, [7, 11]],
 'ayews': [7, 5],
 'yewsat': [7, 6],
 'ta': [7, 11, [7, 11]],
 'syews': [7, 5],
 'yewsst': [7, 6],
 'ts': [7, 11, [7, 11]],
 'odyews': [7, 4],
 'oyewsdt': [7, 5],
 'yewsotd': [7, 6],
 'tod': [7, 11, [7, 11]],
 'oayews': [7, 4],
 'oyewsat': [7, 5],
 'yewsota': [7, 6],
 'toa': [7, 11, [7, 11]],
 'osyews': [7, 4],
 'oyewsst': [7, 5],
 'yewsots': [7, 6],
 'tos': [7, 11, [7, 11]],
 'doyews': [7, 4],
 'dyewsot': [7, 5],
 'yewsdto': [7, 6],
 'tdo': [7, 11, [7, 11]],
 'dayews': [7, 4],
 'dyewsat': [7, 5],
 'yewsdta': [7, 6],
 'tda': [7, 11, [7, 11]],
 'dsyews': [7, 4],
 'dyewsst': [7, 5],
 'yewsdts': [7, 6],
 'tds': [7, 11, [7, 11]],
 'aoyews': [7, 4],
 'ayewsot': [7, 5],
 'yewsato': [7, 6],
 'tao': [7, 11, [7, 11]],
 'adyews': [7, 4],
 'ayewsdt': [7, 5],
 'yewsatd': [7, 6],
 'tad': [7, 11, [7, 11]],
 'asyews': [7, 4],
 'ayewsst': [7, 5],
 'yewsats': [7, 6],
 'tas': [7, 1

<IPython.core.display.Javascript object>

In [194]:
a = list("hello")

<IPython.core.display.Javascript object>